In [1]:
import pandas as pd
from IPython.display import HTML
import math

In [2]:
import inflect
p = inflect.engine()
p.number_to_words(99)

In [ ]:
import urllib.request
from urllib.error import HTTPError
import sys

In [3]:
sheet_id = '1VaF5jAauYYYKo1WH40kLlsQ0-azIX5eMZoFYv5JrVr0'
sheet_names = ['papers','dataset','algorithms','masterThesis','phdThesis']

In [4]:
def write_html(string,filename='filename',extension='pug'):
    Html_file= open(f"{filename}.{extension}","w")
    Html_file.write(string)
    Html_file.close()

## Papers

In [5]:
sheet_name= sheet_names[0]
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'

In [6]:
papers = pd.read_csv(url,header=0)
papers.columns

In [7]:
aut= []
for i,el in papers.iterrows():
    l = [x.strip() for x in el.Authors.split(',')]
    aut.append(l)
flat_list = sum(aut, [])
authorsU =list(set(flat_list))
print(authorsU, len(authorsU))

In [8]:
if '1' in (papers['Research line'][0]).split(','):
    print('eccolo')

In [9]:
lines = [*range(1,6,1)]
lines

In [12]:
papers[papers['Research line'].str.contains('1', case=False, na=False)]

## Visualization order of the papers

In [13]:
papers=papers.sort_values(by=['featured','Year of publication'], ascending = False).reset_index(drop=True)

In [14]:
bullets='<ul>'
for i, row in papers.iterrows():
    authors= row['Authors']
#     pubblicationYear = int(row['Year of publication'])
    pubYearString = row['Year of publication']
    pubblicationYear =  '.' if pubYearString!=pubYearString else ' - '+str(int(pubYearString))+' -'
    title=row['Title of the scientific publication'].strip('.')
    
    journalString=row['Title of the journal or equivalent']
    journal='.' if journalString!=journalString else ', '+journalString.strip()+'.'
    DOI=row['DOI']
    
    bullet=f"<li><strong><a href='https://doi.org/{DOI}' target='_blank'>{title}</a></strong>{pubblicationYear} {authors} {journal}</li>"
    if row['visible on website'] == 'YES':
        bullets+=bullet
bullets+='</ul>'
display(HTML(bullets))

### DOI TO BibTex

In [52]:
doi='10.1007/978-3-030-10997-4_13'
BASE_URL = 'http://dx.doi.org/'
url = BASE_URL + doi
req = urllib.request.Request(url)
req.add_header('Accept', 'application/x-bibtex')
try:
    with urllib.request.urlopen(req) as f:
        bibtex = f.read().decode()
    print(bibtex)
except HTTPError as e:
    if e.code == 404:
        print('DOI not found.')
    else:
        print('Service unavailable.')
    sys.exit(1)

print(type(bibtex))
print(url)


In [54]:
bibs=[]
BASE_URL = 'http://dx.doi.org/'
for i,doi in enumerate(papers.DOI.to_list()):
    if type(doi)==str:
        url = f'{BASE_URL + doi}'
        req = urllib.request.Request(url)
        req.add_header('Accept', 'application/x-bibtex')
        try:
            with urllib.request.urlopen(req) as f:
                bibtex = f.read().decode()
            bibs.append(bibtex)
            # print(bibtex)
        except HTTPError as e:
            print('error')
            # if e.code == 404:
            #     print('DOI not found.')
            # else:
            #     print('Service unavailable.')
            # # sys.exit(1)

        print(bibtex)
bibs

In [55]:
bibs[0]

In [56]:
with open("xai_papers.bib", "w") as file:
    for item in bibs:
        file.write(item + "\n")

### Pug version

In [14]:
bullets=''
for i, row in papers.iterrows():
    authors= row['Authors']
#     pubblicationYear = int(row['Year of publication'])
    pubYearString = row['Year of publication']
    pubblicationYear =  '.' if pubYearString!=pubYearString else str(int(pubYearString))
    
    title=row['Title of the scientific publication'].strip('.')
    
    journalString=row['Title of the journal or equivalent']
    journal='.' if journalString!=journalString else ', '+journalString.strip()+'.'
    DOI=row['DOI']
    
    bullet=f'''li
    div.mb-3
        a(href='https://doi.org/{DOI}', target="_blank")
            | {authors} ({pubblicationYear}). #[strong {title}]
'''
    if row['visible on website'] == 'YES':
        bullets+=bullet
# display(HTML(bullets))

In [15]:
display(HTML(bullets))

In [506]:
write_html(bullets, filename='paper-list')

In [17]:
cards=''
for i, row in papers.iterrows():
    authors= row['Authors']
    abstract= row['abstract']
    researchLine= row['Research line']
    idAlpha = row['Id di Riccardo']
    
    if isinstance(researchLine, str):
        researchLine=researchLine.replace(",", "▪")
        rlString=f'''p.my-1
                small Research Line #[strong {researchLine}]'''
    else:
        rlString=''

        
        
        
    pubYearString = row['Year of publication']
    pubblicationYear =  '.' if pubYearString!=pubYearString else str(int(pubYearString))
    
    title=row['Title of the scientific publication'].strip('.')
    
    container=row['container']
    if not isinstance(container, str):
        container=''
    else:
        container=f' - {container}'
    
#     journalString=row['Title of the journal or equivalent']
#     journal='.' if journalString!=journalString else ', '+journalString.strip()+'.'
    journal=row['Title of the journal or equivalent']
    if not isinstance(journal, str):
        journal=''
    else:
        journal=f'. In {journal}'
        
    DOI=row['DOI']
    doiLink=row['doi link']
    image= row['image file']
    
    if isinstance(DOI, str):
        #Doi to bibtex
        BASE_URL = 'http://dx.doi.org/'
        url = BASE_URL + DOI
        req = urllib.request.Request(url)
        req.add_header('Accept', 'application/x-bibtex')
        try:
            with urllib.request.urlopen(req) as f:
                bibtex = f.read().decode()
                bibtex=bibtex.replace('\t','                    ')
                bibtex=bibtex[:-1] + "                }"
#             print(bibtex)
        except HTTPError as e:
            if e.code == 404:
                bibtex='BibTex not found'
            else:
                bibtex='BibTex not found.'# Service unavaillable
#             sys.exit(1)
    else:
        bibtex='BibTex not found'
    
    #ADD BUTTONS IF:    
        #More information
    if (isinstance(abstract, str) or (bibtex not in['BibTex not found','BibTex not found.']) ):
        if not (isinstance(abstract, str)):
            abstract= 'ABSTRACT NOT FOUND'
        
        abstractButton= f'''p.my-1
            a.btn-mini.px-2.btn-secondary.small(href='#' data-toggle='collapse' data-target='#collapse-{p.number_to_words(i)}' aria-expanded='true' aria-controls='collapseAbs') More information'''
    else:
        abstractButton=''
        
        #External link
    if (isinstance(doiLink, str)):
        externalButton= f'''p.my-1
            a.btn-mini.px-2.btn-secondary.small(href='{doiLink}', target="_blank") External Link'''
    else:
        externalButton= ''

    
    
    
    
    ptOne=f'''#{idAlpha}.row.mt-4.justify-content-center
    .col-lg-1.text-right
        h4 {i+1}.
        p [{idAlpha}]'''
    # summary image?
    if row['summary image']=='YES':
        ptTwo=f'''
    .col-lg-2.pl-0
        img(src='assets/images/publication/{image} ' alt="immagine" style='width:100%;' data-toggle='modal' data-target='#modal-{p.number_to_words(i)}' type='button').mr-3
        #modal-{p.number_to_words(i)}.modal.fade(tabindex='-1' role='dialog' aria-labelledby='#modal-{p.number_to_words(i)}-Title' aria-hidden='true')
            .modal-dialog.modal-lg.modal-dialog-centered(role='document')
                .modal-content
                    .modal-header
                        p.small {title}
                        button.close(type='button' data-dismiss='modal' aria-label='Close')
                            span(aria-hidden='true') &times;
                    .modal-body
                        img(src='assets/images/publication/{image} ' alt="immagine" )
    .col-lg-6.bg-yellow.p-3'''
    else:
        ptTwo=f'''
    .col-lg-8.bg-yellow.p-3'''

    ptThree=f'''
        #accordion-{p.number_to_words(i)}.accordion
        | #[strong {title}] 
        br
        | #[em {authors}] ({pubblicationYear}){container}{journal}
        #collapse-{p.number_to_words(i)}.collapse(aria-labelledby='heading-{p.number_to_words(i)}' data-parent='#accordion-{p.number_to_words(i)}')
            div.bg-yellow
                hr
                p.small #[strong Abstract]
                p.small {abstract}
            p.small.pt-2 #[strong BibTex]
            p.small.
                {bibtex}
    .col-lg-2.pl-3
        {abstractButton}
        {externalButton}
        {rlString}
'''
    
    card= ptOne+ptTwo+ptThree
    if row['visible on website'] == 'YES':
        cards+=card
    
# display(HTML(bullets))

In [18]:
display(HTML(cards))

In [510]:
write_html(cards, filename='paper-cards')

## Thesis

In [36]:
sheet_name= sheet_names[3]
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'

In [37]:
masterThesis = pd.read_csv(url)
masterThesis.columns

In [38]:
masterThesis

In [52]:
masterThesis=masterThesis.sort_values(['Thesis ','Status [Ongoing] / [completed]'],ascending=[False,True]).reset_index(drop=True)

In [53]:
bulletsThesis='<ul>'

for i, row in masterThesis.iterrows():
    author= row['Author']
    title= row['Title']
    title = '' if title!=title else title
    
    status= row['Status [Ongoing] / [completed]']
    
    classification = row['Thesis ']
    year= row['Year']
    bullet=f"<li>{author}, <strong>{title}</strong> [{classification} - {year} - {status}]</li>"
    if row['visible on website'] == 'YES':
        bulletsThesis+=bullet
bulletsThesis+='</ul>'
display(HTML(bulletsThesis))

In [32]:
bulletsThesis

#### Pug version

In [51]:
bulletsThesis=''
for i, row in masterThesis.iterrows():
    author= row['Author']
    title= row['Title']
    title = '' if title!=title else title
    
    status= row['Status [Ongoing] / [completed]']
    
    classification = row['Thesis ']
    year= row['Year']
    
    bullet=f'''li
    div.mb-3
        | {author} - #[strong {title}]. [{classification} - {year} - {status}]
'''
    if row['visible on website'] == 'YES':
        bulletsThesis+=bullet
    
#     bullet=f"<li>{author}, <strong>{title}</strong> [{classification} - {year} - {status}]</li>"
display(HTML(bulletsThesis))

In [19]:
write_html(bulletsThesis, filename='thesis-list')

# Algorithms made by XAI group 

In [20]:
sheet_name= sheet_names[2]
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'

In [21]:
algo = pd.read_csv(url)
algo.columns

In [22]:
bulletsAlgo='<ul>'

for i, row in algo.iterrows():
    author= row['Authors']
    title= row['Title']
    title = '' if title!=title else title
    
    description= row['Brief Description']
    description = '' if description!=description else ' - '+description.strip('.')+', '
    
    githubLink=row['github link']
    
    bullet=f"<li><strong>{title}</strong> {description} [ {author} ] </li>"
    bulletsAlgo+=bullet
bulletsAlgo+='</ul>'
display(HTML(bulletsAlgo))